In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
import numpy as np


# Step 1: Load the training data
train_url = ("https://github.com/dustywhite7/Econ8310/raw/master/AssignmentData/assignment3.csv")
data = pd.read_csv(train_url)

# Step 2: Define features (X) and target (y)
X = data.drop(columns=["meal"])  # Drop the target variable
y = data["meal"]  # Define the target variable

# Check and preprocess the data
# Identify categorical columns
categorical_columns = X.select_dtypes(include=["object"]).columns

# Apply one-hot encoding to categorical columns
X = pd.get_dummies(X, columns=categorical_columns, drop_first=True)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

# Step 3: Train the Random Forest model
model = RandomForestClassifier(n_estimators=1000, max_depth=20)  # Instantiate the model
modelFit = model.fit(X_train, y_train)  # Fit the model to training data

# Step 4: Evaluate the model
y_pred_val = modelFit.predict(X_val)  # Generate predictions on validation data
print("Validation Accuracy:", accuracy_score(y_val, y_pred_val))  # Print accuracy

# Step 5: Load the test data
test_url = "https://github.com/dustywhite7/Econ8310/raw/master/AssignmentData/assignment3test.csv"
test_data = pd.read_csv(test_url)

# Align test data with training data columns
test_data_aligned = pd.get_dummies(test_data)
test_data_aligned = test_data_aligned.reindex(columns=X_train.columns, fill_value=0)

# Generate predictions
predict = modelFit.predict(test_data_aligned)

# Save predictions as a series
pred = pd.Series(modelFit.predict_proba(test_data_aligned)[:, 1], name="Predictions")
print(pred)  # Output predictions



Validation Accuracy: 0.8320395201129146
0      0.190945
1      0.167840
2      0.167840
3      0.156453
4      0.196327
         ...   
995    0.167840
996    0.155746
997    0.167840
998    0.167279
999    0.167840
Name: Predictions, Length: 1000, dtype: float64
Tjur R-squared: nan


C:\Users\zkpet\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\zkpet\anaconda3\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


AssertionError: Tjur R-squared is below 0.05. Improve your model.